In [2]:
%matplotlib inline
 
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.optim import Adam
import networkx as nx

In [ ]:
# https://www.youtube.com/watch?v=-UjytpbqX4A&t=15s -> 1:10:30

### Challenges of Graph Generation: 

- A Graph with n nodes can be represented by up to n! equivalent adjacency matrices, each corresponding to a different, arbitrary node ordering. Given that a graph can have multiple representations, it is difficult for the models to calculate
the distance between the generated graphs and groundtruth graphs white training. 
Thus it may require us to design
either a **pre-defined node ordering** or a **node permutation
invariant reconstruction objective function**.

- Two nodes are more likely to be connected if they
share common neighbors. Therefore, the generation of each
node or edge **cannot** be modeled as an independent event

- Large Output space due to n² adjacency matrix 

- Reconstructing the desecrate graph objects (i.e., nodes and
edges) from continuous spaces results into different graph
decoder process, such as sequentially generating the nodes
of the graphs or generating the adjacent matrix of graphs in
one-shot

In [6]:
def create_small_graph():
    G = nx.karate_club_graph()  # Creating a small example graph
    edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
    x = torch.eye(G.number_of_nodes(), dtype=torch.float)  # One-hot encoding of node features
    data = Data(x=x, edge_index=edge_index)
    return data

data = create_small_graph()
print(type(data))
print(data.x)
print(data.edge_index)

<class 'torch_geometric.data.data.Data'>
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
torch.Size([2, 78])


##### Standard Decoder Architecture: 
compute $\tilde{A}_{i,j}$ based on the inner-product operations of two node embedding $Z_i$
and $Z_j$ . This reflects the idea that nodes that are close in
the embedding space should have a high probability of
being connected

In [7]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        '''
        :param input_dim: number of node_features
        :param hidden_dim: For Convolution: latent representation of node features 
        :param latent_dim: Dimension of latent embedding z 
        '''
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim) # first layer 
        # Layers for GCN_mu and GCN_sigma
        self.conv_mu = GCNConv(hidden_dim, latent_dim)
        self.conv_logvar = GCNConv(hidden_dim, latent_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        mu = self.conv_mu(x, edge_index)
        # The reason for logvar, is because it yields negative and positive values 
        logvar = self.conv_logvar(x, edge_index)
        return mu, logvar


The encoder (inference model) of VGAE consists of graph convolutional networks (GCNs). It takes an adjacency
matrix A and a feature matrix X as inputs and generates the latent variable Z as output. The first GCN layer
generates a lower-dimensional feature matrix. It is defined as <br>

$$\( \bar{X} \) = GCN(X, A) = ReLU(\( \tilde{A} \)XW_0)$$

$ A \in R^{nxn}$ and $X \in R^{nxf}$, where X is $H^{(0)}$ and $W \in R^{F^{(l)}xF^{(l+1)}}$

A-tilde is the symmetrically normalized adjacency matrix.

$$\tilde{A} = \( D^{-1/2} \)A\( D^{-1/2} \) $$

The second GCN layer generates μ and logσ², where
$$\mu = GCN_{\mu}(X,A) = \( \tilde{A} \)\( \bar{X} \)W_{1}  $$
$$log\sigma² = GCN_{\sigma}(X,A) = \( \tilde{A} \)\( \bar{X} \)W_{1}  $$
Both $\mu$ and log$\sigma$ share the same computation 

Now if we combine the math of two-layer GCN together, we get
...
which generates μ and logσ².

Then we can calculate Z using parameterization trick

$$ Z = \mu + \sigma * \epsilon $$

where ε ~ N(0,1).

In [8]:
class GCNDecoder(torch.nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        ''' Better to only use inner Product layers (MLP)?'''
        super(GCNDecoder, self).__init__()
        self.conv1 = GCNConv(latent_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, z, edge_index):
        x = F.relu(self.conv1(z, edge_index))
        x = torch.sigmoid(self.conv2(x, edge_index))
        return x

class VGAE(torch.nn.Module):
    def __init__(self, encoder, decoder):
        super(VGAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar) # why *  0.5? 
            eps = torch.randn_like(std)
            return mu + eps * std
        else:
            return mu

    def forward(self, x, edge_index):
        mu, logvar = self.encoder(x, edge_index)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decoder(z, edge_index)
        return recon_x, mu, logvar


In [11]:
def generate_synthetic_graph(num_nodes):
    model.eval()
    with torch.no_grad():
        z = torch.randn(num_nodes, latent_dim).to(device)
        edge_index = torch.combinations(torch.arange(num_nodes), r=2).t().to(device)
        recon_x = model.decoder(z, edge_index)
        return recon_x

# Generate a synthetic graph with the same number of nodes as the small graph
synthetic_graph = generate_synthetic_graph(data.num_nodes)
# Convert the output to an adjacency matrix
synthetic_adj_matrix = (synthetic_graph > 0.5).float()

The degree to which the Auto Encoder simply reproduces the graph, depends on how small the **difference between the number of hidden dimensions and the number of input dimensions** is.

In [9]:
# Configurate: 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model parameters for a smaller dataset
input_dim = data.num_node_features
hidden_dim = 16  # Adjusted hidden dimension
latent_dim = 8   # Adjusted latent dimension

# Initialize models with adjusted dimensions
encoder = GCNEncoder(input_dim, hidden_dim, latent_dim).to(device)
decoder = GCNDecoder(latent_dim, hidden_dim, input_dim).to(device)
model = VGAE(encoder, decoder).to(device)

# Optimizer
optimizer = Adam(model.parameters(), lr=0.01)

def train(data):
    model.train()
    optimizer.zero_grad()
    recon_x, mu, logvar = model(data.x, data.edge_index)
    loss = model.loss_function(recon_x, data.x, mu, logvar)
    loss.backward()
    optimizer.step()
    return loss.item()

# Define the loss function
def loss_function(recon_x, x, # For BCE
                  mu, logvar): # FOR KLD 
    '''PyTorch dynamically builds and traverses the computational graph, applying the chain rule to compute gradients for all operations involved
    Each tensor operation (torch.sum, logvar.pow(2), logvar.exp(), etc.) has a known gradient. PyTorch tracks these operations and builds a backward function that can compute the gradient for each of these operations.'''
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) # should be legit 
    return BCE + KLD

model.loss_function = loss_function

# Load data
data = data.to(device)

In [10]:
# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    # What is an epoch? one node? 
    loss = train(data)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')


Epoch 0, Loss: 782.3570
Epoch 10, Loss: 482.3631
Epoch 20, Loss: 254.7292
Epoch 30, Loss: 235.6179
Epoch 40, Loss: 201.7655
Epoch 50, Loss: 194.7714
Epoch 60, Loss: 183.9261
Epoch 70, Loss: 172.0984
Epoch 80, Loss: 171.5353
Epoch 90, Loss: 167.1517
Epoch 100, Loss: 174.7078
Epoch 110, Loss: 179.3512
Epoch 120, Loss: 160.6138
Epoch 130, Loss: 167.1098
Epoch 140, Loss: 161.7437
Epoch 150, Loss: 163.6620
Epoch 160, Loss: 162.6941
Epoch 170, Loss: 156.4730
Epoch 180, Loss: 156.8359
Epoch 190, Loss: 161.4288


In [12]:
torch.randn(data.num_nodes, latent_dim)

tensor([[-1.2222, -0.3827, -0.8971, -0.2272,  0.1467,  0.3660,  0.8537, -0.9697],
        [-0.1939, -1.0909, -0.8521,  0.4935,  0.3323, -0.3392, -2.7095, -0.3395],
        [-0.3465,  0.7839, -1.3669,  0.9866, -1.5625, -0.0280,  1.8097,  0.2726],
        [-0.1660,  0.8072,  1.0152,  0.0068,  1.6382, -0.1477, -0.5107,  0.8677],
        [ 0.2184, -0.5799,  0.5228,  0.5028, -1.0484, -0.9361, -1.8016, -0.6611],
        [-0.1086, -0.8943, -0.4106, -0.0602, -0.2080, -0.3285, -1.1321,  1.7652],
        [-1.1012,  0.5215,  0.5605,  1.5581,  0.2170, -1.3616, -0.7893,  0.4434],
        [-0.9681,  0.4497, -0.4210,  0.1436, -0.1801,  0.7429, -1.4777,  0.7788],
        [-0.2805,  0.6716, -0.6382,  1.0572,  0.3380,  1.0947,  0.8290, -0.7078],
        [-0.5782, -1.3198,  0.0367, -0.1153, -0.2776,  0.5019, -0.2993, -0.6841],
        [ 1.6371,  0.0761,  1.3484,  0.2744, -0.5625,  0.0747, -1.0176, -0.5474],
        [ 0.0748, -0.3228,  0.1408,  1.1832, -1.2772,  0.0453,  0.9017, -0.2338],
        [ 0.1047

#### New Approach 

In [24]:
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T
from torch_geometric.datasets import ZINC
from torch_geometric.nn import VGAE, GCNConv
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [25]:
transform = T.Compose([
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      split_labels=True, add_negative_train_samples=True)
])

#Combines a single transformation step using T.Compose.
#The T.RandomLinkSplit transformation:
#Splits the graph's edges into training (85%), validation (5%), and test (10%) sets.
#Treats the graph as undirected.
#Generates labels for the split edges to indicate whether they are positive or negative samples.
#Adds negative samples to the training set to facilitate link prediction tasks.

dataset = ZINC(root='/tmp/ZINC', subset=True, transform=transform)
train_data_list, val_data_list, test_data_list = [], [], []
for train_data, val_data, test_data in dataset:
    try:
        if val_data.neg_edge_label is not None:
            train_data.x = F.normalize(train_data.x.float())
            val_data.x = F.normalize(val_data.x.float())
            test_data.x = F.normalize(test_data.x.float())
            train_data_list.append(train_data)
            val_data_list.append(val_data)
            test_data_list.append(test_data)
    except:
        continue

In [23]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)


In [18]:
in_channels, out_channels, lr, n_epochs = dataset.num_features, 16, 1e-2, 5 # num_features = 1
gen_graphs, threshold, batch_size, add_self_loops = 5, 0.5, 2, True
model = VGAE(VariationalGCNEncoder(in_channels, out_channels))
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_loader = DataLoader(train_data_list, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data_list, batch_size=batch_size)